In [2]:
import numpy as np

def random_initialization(A, rank):
    """
    Initialize matrices W and H randomly.

    Parameters:
    - A: Input matrix
    - rank: Rank of the factorization

    Returns:
    - W: Initialized W matrix
    - H: Initialized H matrix
    """
    num_docs = A.shape[0]
    num_terms = A.shape[1]
    W = np.random.uniform(1, 2, (num_docs, rank))
    H = np.random.uniform(1, 2, (rank, num_terms))
    return W, H

def nndsvd_initialization(A, rank):
    """
    Initialize matrices W and H using Non-negative Double Singular Value Decomposition (NNDSVD).

    Parameters:
    - A: Input matrix
    - rank: Rank of the factorization

    Returns:
    - W: Initialized W matrix
    - H: Initialized H matrix
    """
    u, s, v = np.linalg.svd(A, full_matrices=False)
    v = v.T
    w = np.zeros((A.shape[0], rank))
    h = np.zeros((rank, A.shape[1]))

    w[:, 0] = np.sqrt(s[0]) * np.abs(u[:, 0])
    h[0, :] = np.sqrt(s[0]) * np.abs(v[:, 0].T)

    for i in range(1, rank):
        ui = u[:, i]
        vi = v[:, i]
        ui_pos = (ui >= 0) * ui
        ui_neg = (ui < 0) * -ui
        vi_pos = (vi >= 0) * vi
        vi_neg = (vi < 0) * -vi

        ui_pos_norm = np.linalg.norm(ui_pos, 2)
        ui_neg_norm = np.linalg.norm(ui_neg, 2)
        vi_pos_norm = np.linalg.norm(vi_pos, 2)
        vi_neg_norm = np.linalg.norm(vi_neg, 2)

        norm_pos = ui_pos_norm * vi_pos_norm
        norm_neg = ui_neg_norm * vi_neg_norm

        if norm_pos >= norm_neg:
            w[:, i] = np.sqrt(s[i] * norm_pos) / ui_pos_norm * ui_pos
            h[i, :] = np.sqrt(s[i] * norm_pos) / vi_pos_norm * vi_pos.T
        else:
            w[:, i] = np.sqrt(s[i] * norm_neg) / ui_neg_norm * ui_neg
            h[i, :] = np.sqrt(s[i] * norm_neg) / vi_neg_norm * vi_neg.T

    return w, h

def multiplicative_update(A, k, max_iter, init_mode='random'):
    """
    Perform Multiplicative Update (MU) algorithm for Non-negative Matrix Factorization (NMF).

    Parameters:
    - A: Input matrix
    - k: Rank of the factorization
    - max_iter: Maximum number of iterations
    - init_mode: Initialization mode ('random' or 'nndsvd')

    Returns:
    - W: Factorized matrix W
    - H: Factorized matrix H
    - norms: List of Frobenius norms at each iteration
    """
    if init_mode == 'random':
        W, H = random_initialization(A, k)
    elif init_mode == 'nndsvd':
        W, H = nndsvd_initialization(A, k)

    norms = []
    epsilon = 1.0e-10
    for _ in range(max_iter):
        # Update H
        W_TA = W.T @ A
        W_TWH = W.T @ W @ H + epsilon
        H *= W_TA / W_TWH

        # Update W
        AH_T = A @ H.T
        WHH_T = W @ H @ H.T + epsilon
        W *= AH_T / WHH_T

        norm = np.linalg.norm(A - W @ H, 'fro')
        norms.append(norm)

    return W, H, norms


Let's try out the methods.

In [3]:
import numpy as np
# Generate a random matrix A
A = np.random.rand(10, 10)

# Test random_initialization
rank = 3
W_rand, H_rand = random_initialization(A, rank)
print("Random Initialization:")
print("W:\n", W_rand)
print("H:\n", H_rand)
print()

# Test nndsvd_initialization
W_nndsvd, H_nndsvd = nndsvd_initialization(A, rank)
print("NNDSVD Initialization:")
print("W:\n", W_nndsvd)
print("H:\n", H_nndsvd)
print()

# Test multiplicative_update
max_iter = 100
W_mu, H_mu, norms = multiplicative_update(A, rank, max_iter)
print("Multiplicative Update:")
print("W:\n", W_mu)
print("H:\n", H_mu)
print("Frobenius Norms:\n", norms)


Random Initialization:
W:
 [[1.60127714 1.61578803 1.10252938]
 [1.80994344 1.19190118 1.70173218]
 [1.35775344 1.63257136 1.55256816]
 [1.45241423 1.13703559 1.63997031]
 [1.25390336 1.8220465  1.8431239 ]
 [1.07861354 1.75613709 1.97168692]
 [1.77838964 1.31662462 1.06008103]
 [1.14146372 1.70502738 1.50399734]
 [1.70704622 1.67622731 1.23146759]
 [1.47146038 1.92612213 1.01511865]]
H:
 [[1.40486812 1.18437235 1.5244241  1.8001452  1.62829776 1.44190436
  1.24303811 1.022523   1.73029604 1.27300573]
 [1.1745511  1.9650149  1.16546641 1.93672905 1.16764288 1.0554667
  1.78021395 1.39417361 1.34054031 1.77218664]
 [1.83844961 1.8050167  1.50570834 1.55624704 1.58455449 1.24552429
  1.44751868 1.17380593 1.34685091 1.337465  ]]

NNDSVD Initialization:
W:
 [[ 6.96191656e-01  8.73716553e-02  5.44999075e-01]
 [ 8.54777426e-01  1.15916128e-01 -0.00000000e+00]
 [ 7.09543403e-01  3.48335204e-04 -0.00000000e+00]
 [ 7.50811005e-01 -0.00000000e+00  5.98833837e-02]
 [ 6.83101927e-01 -0.00000000e+